#### 유튜브 : https://www.youtube.com/watch?v=xMPkI-GHlUM  
#### 피드백은 rlaalstn1504@naver.com 여기로 주시면 감사하겠습니다~

1. 코드 실행시간은 1분으로 제한되어 있음 ㅡ> 계산량이 많은 하이퍼파라미터 최적화는 어려움 
2. 높은 정확도를 내는 다양한 테크닉보다는 빠르고 손쉽게 사용가능한 방법 위주로 사용 

#### -코드가 생각나지 않거나, 시간이 부족할 때?

- 분류 : label 중 비중이 더 많은 것으로 모두 찍어서 제출    
- 회귀 : 사용 가능한 일부 변수만이라도 사용해서 예측 후 제출

-  실기환경 : https://www.dataq.or.kr/www/board/view.do?bbsKey=eyJiYnNhdHRyU2VxIjoxLCJiYnNTZXEiOjUwOTM0M30=&boardKind=notice

### * 고객 3,500명에 대한 학습용 데이터(y_train.csv, X_train.csv)를 이용하여 성별예측 모형을 만든 후, 이를 평가

### 용 데이터(X_test.csv)에 적용하여 얻은 2,482명 고객의 성별 예측값(남자일 확률)을 

### 다음과 같은 형식의 CSV 파일로 생성하시오.(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점) 

### DataFrame.to_csv("0000.csv", index=False)

In [2]:
import pandas as pd 
from xgboost.sklearn import XGBClassifier # 분류
from xgboost.sklearn import XGBRegressor  # 회귀
train_x = pd.read_csv('data/X_train.csv',encoding='cp949') 
train_y = pd.read_csv('data/y_train.csv',encoding='cp949')
test_x = pd.read_csv('data/X_test.csv',encoding='cp949')

In [3]:
train_x.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [4]:
train_x['환불금액'] = train_x['환불금액'].fillna(0) 
test_x['환불금액'] = test_x['환불금액'].fillna(0)

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
lab1 = LabelEncoder() 
lab1.fit(train_x['주구매상품']) 
train_x['주구매상품'] = lab1.transform(train_x['주구매상품'])  
test_x['주구매상품'] = lab1.transform(test_x['주구매상품'])

lab2 = LabelEncoder() 
lab2.fit(train_x['주구매지점']) 
train_x['주구매지점'] = lab2.transform(train_x['주구매지점'])  
test_x['주구매지점'] = lab2.transform(test_x['주구매지점'])

In [7]:
from xgboost.sklearn import XGBClassifier 
from sklearn.model_selection import KFold  
from sklearn.model_selection import cross_val_score

In [8]:
k_fold = KFold(shuffle=True, random_state = 0) 
xgb = XGBClassifier()

In [9]:
cross_val_score(xgb,train_x.drop('cust_id',axis=1),train_y['gender'],cv=k_fold,scoring='accuracy').mean()

0.6222857142857142

In [10]:
xgb.fit(train_x.drop('cust_id',axis=1),train_y['gender'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
xgb.predict_proba(test_x.drop('cust_id',axis=1))[:,1] # [:,1]은 전체 2개로 이루어진 개별값들 중 두번째값만 보겠다는 것입니다.

array([0.18315685, 0.16423059, 0.05545351, ..., 0.48270735, 0.46732926,
       0.66104853], dtype=float32)

In [12]:
test_x['gender'] = xgb.predict_proba(test_x.drop('cust_id',axis=1))[:,1] 

In [13]:
test_x

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,3500,70900400,22000000,4050000.0,3,9,13,1.461538,0.789474,26,0.183157
1,3501,310533100,38558000,48034700.0,9,19,90,2.433333,0.369863,3,0.164231
2,3502,305264140,14825000,30521000.0,0,8,101,14.623762,0.083277,3,0.055454
3,3503,7594080,5225000,0.0,32,9,5,2.000000,0.000000,47,0.396385
4,3504,1795790,1411200,0.0,20,22,3,2.666667,0.125000,8,0.114526
...,...,...,...,...,...,...,...,...,...,...,...
2477,5977,82581500,23976000,0.0,3,9,8,1.750000,0.642857,40,0.828178
2478,5978,480000,480000,0.0,17,2,1,1.000000,0.000000,0,0.705911
2479,5979,260003790,25750000,0.0,6,8,19,3.736842,0.915493,18,0.482707
2480,5980,88991520,18120000,0.0,27,8,5,3.600000,0.444444,60,0.467329


In [14]:
test_x.to_csv("0000.csv", index=False)